# Vagrant & Virtualbox

Vagrant와 Virtualbox를 이용한 개발환경

Vagrant는 설정파일을 통해 Virtualbox에서 Virtual Network상에 다수의 Virtual Machine을 빠르게 생성/삭제할 수 있는 편리한 툴입니다. 다수의 노드를 통해서 플랫폼의 설치 및 자동화를 개발/테스트해야 하는 경우 테스트만으로도 비용이 만만치 않습니다. 이럴 경우 성능 좋은 개발용 싱글머신에서 Vagrant와 Virtualbox를 이용할 경우 시간과 비용 모두를 절약할 수 있습니다.

## Virtualbox 설치

https://www.virtualbox.org/ 

### Ubuntu

Virtualbox 의 Repository 를 APT 소스 리스트에 추가하고 Virtualbox 최신버전을 설치합니다.

```bash
wget -q https://www.virtualbox.org/download/oracle_vbox_2016.asc -O- | sudo apt-key add -
wget -q https://www.virtualbox.org/download/oracle_vbox.asc -O- | sudo apt-key add -
sudo add-apt-repository "deb [arch=amd64] http://download.virtualbox.org/virtualbox/debian $(lsb_release -cs) contrib"
sudo apt update
sudo apt install -y virtualbox virtualbox-ext-pack
```

### MaxOS

https://www.virtualbox.org/ 홈페이지를 방문하여 "[Downloads](https://www.virtualbox.org/wiki/Downloads)" 메뉴에서 "OS X hosts" 링크를 통해 dmg 파일을 다운로드 받아 설치합니다.

## Vagrant 설치

https://www.vagrantup.com/docs/index

### Ubuntu

Vagrant 최신버전을 다운받아 설치합니다. Vagrant 의 설정파일에서 Disksize를 조정할 수 있도록 vagrant-disksize 플러그인도 설치해 줍니다.

```bash
wget -c https://releases.hashicorp.com/vagrant/2.0.3/vagrant_2.0.3_x86_64.deb -P /tmp
sudo dpkg -i /tmp/vagrant_2.0.3_x86_64.deb
vagrant plugin install vagrant-disksize
```

### MacOS

https://www.vagrantup.com/ 홈페이지를 방문하여 "[Downloads](https://www.vagrantup.com/downloads.html)" 메뉴에서 "macOS" 용 dmg 파일을 다운로드 받아 설치합니다.

터미널 창을 통해서 vagrant-disksize 플러그인을 설치합니다.

```bash
vagrant plugin install vagrant-disksize
```

## Vagrantfile

Vagrant는 Vagrantfile를 이용하여 다양한 가상 환경을 설정할 수 있습니다. vagrant를 위한 디렉토리를 생성 후 초기화 명령을 수행하면 기본 Vagrantfile 이 생성되며, 이 파일을 용도에 맞게 수정하여 사용하면 됩니다.

Vagrant를 통해 생성되는 VM 들은 기본적으로 암호 없이 sudo를 수행할 수 있는 vagrant 계정이 생성됩니다. VM 생성시 SSH 로그인 옵션이 Key기반이기 때문에 vagrant를 통해 랜덤하게 생성된 private_key를 통해 접속해야 합니다. 암호를 사용한 접근은 vagrant 계정이 기본 암호를 사용하기 때문에 추천되지 않습니다. VM 에서 사용하는 네트워크도 public_network / private_network 로 구분하여 사용할 수 있습니다. private_network를 사용할 경우 해당 VM 은 로컬 머신에서만 접속할 수 있기 때문에 public_network를 주로 사용하며, bridge 옵션을 이용하여 호스트 머신의 네트워크 카드를 공유하여 IP 공유기의 내부 IP 를 받아 사용함으로써 좀 더 실 환경과 비슷한 테스트를 진행할 수 있습니다.  

아래는 Single-VM 과 Multi-VM 구성 예 입니다.

### Single-VM 예

vagrant 초기화

```bash
mkdir single
cd single
vagrant init
```

설정파일 Vagrantfile의 예

```ruby
# -*- mode: ruby -*-
# vi: set ft=ruby :

# All Vagrant configuration is done below. The "2" in Vagrant.configure
# configures the configuration version (we support older styles for
# backwards compatibility). Please don't change it unless you know what
# you're doing.

ENV["LC_ALL"] = "en_US.UTF-8"

Vagrant.configure("2") do |config|
  # Use the same key for each machine
  # vagrant 에서는 머신 생성기 각 다른 키를 사용하는데 하나의 키를 사용하도록 설정
  # ansible 같은 원격 관리 툴에서 키를 이용하여 접속할때 편리
  config.ssh.insert_key = false 
    
  config.vm.define "s01" do |host|
    host.vm.box = "ubuntu/bionic64" # Ubuntu 18.04 LTS
    host.vm.hostname = "s01"
    #host.vm.network "private_network", ip: "10.0.1.1" # 로컬 머신에서만 접근 가능      
    #host.vm.network "public_network", ip: "192.168.0.50" # 같은 192.168.0.x 대 네트워크에서만 접근 가능
    #host.vm.network "public_network", bridge: "en0" # 로컬 머신의 en0 네트워크 카드를 공유하여 IP 공유기에서 제공하는 DHCP IP 대역 사용 가능
    host.vm.network "public_network", bridge: "en0", 172.30.1.20 # 로컬 머신의 en0 네트워크 카드를 공유하여 IP 공유기에서 제공하는 IP 대역 사용 가능      
    host.vm.provider "virtualbox" do |vb|
      vb.gui = false
      vb.cpus = 4
      vb.memory = "16384"
    end
    host.disksize.size = '128GB'
    host.vm.synced_folder "../../../", "/home/vagrant/st-kilda-pier", owner: "vagrant", group: "vagrant"
  end

end
```

### Multi-VM 예

vagrant 초기화

```bash
mkdir cluster
cd cluster
vagrant init
```

설정파일 Vagrantfile의 예, loop를 이용하여 여러개의 VM 과 IP를 설정합니다.

```ruby
# -*- mode: ruby -*-
# vi: set ft=ruby :

# All Vagrant configuration is done below. The "2" in Vagrant.configure
# configures the configuration version (we support older styles for
# backwards compatibility). Please don't change it unless you know what
# you're doing.

ENV["LC_ALL"] = "en_US.UTF-8"

n_nodes = 5

Vagrant.configure("2") do |config|

  (1..n_nodes).each do |i|
    config.ssh.insert_key = false 
    config.vm.define "c#{'%02d' % i}" do |host|
      host.vm.box = "ubuntu/bionic64" # Ubuntu 18.04 LTS
      host.vm.hostname = "c#{'%02d' % i}"
      host.vm.network "public_network", bridge: "en0", ip: "172.30.1.5#{i}"
      #host.vm.network "private_network", ip: "10.0.0.#{i}"
      host.vm.provider "virtualbox" do |vb|
        vb.gui = false
        vb.cpus = 4
        vb.memory = "16384"
      end
      host.disksize.size = '64GB'
      host.vm.synced_folder "../../../", "/home/vagrant/st-kilda-pier", owner: "vagrant", group: "vagrant"
    end
  end

end
```

## Vagrant 주요 명령

* vagrant init
  * Vagrnatfile 생성
* vagrant plugin 
  * vagrant plugin 설치/삭제
* vagrant box list
  * box 리스트 출력
* vagrant box add ubuntu/xenial64
  * box 추가, ubuntu/xenial64(ubuntu 16.04)
  * box 이미지는 https://app.vagrantup.com/boxes/search 에서 검색
* vagrant box remove ubuntu/xenial64
  * box 삭제
* vagrant up
  * 가상머신 생성 및 구동
* vagrant ssh-config
  * 가상머신의 ssh 구성 정보 출력 / ssh 에 접속하기 위한 key 위치를 볼 수 있음
* vagrant ssh s01
  * s01 hostname 의 가상머신에 접속
* vagrant reload
  * VM 재부팅
* vagrant halt
  * 가상머신 중지
* vagrant destroy 
  * 가상머신 삭제